In [404]:
import numpy as np
from scipy.spatial.distance import hamming
import tensorflow as tf
from collections import OrderedDict
import time
import matplotlib.pyplot as plt
import copy
import pathlib

In [405]:
error_correction = np.load("table_error_correction_2bit.npy")
print(error_correction)

[-7 -7 -7 -5 -7  5 -2 -1 -7  5  2  3  5  5  6  5 -7 -7  2 -1 -4 -1 -1 -1
  2  1  2  2  4  5  2 -1 -7 -5 -5 -5 -4 -3  6 -5  0  1  6 -5  6  5  6  6
 -4  1 -6 -5 -4 -4 -4 -1  1  1  2  1 -4  1  6  7 -7 -7 -2  3 -2 -3 -2 -2
  0  3  3  3  4  5 -2  3 -7 -7 -6 -7  4 -7 -2 -1  4 -7  2  3  4  4  4  7
  0 -3 -6 -5 -3 -3 -2 -3  0  0  0  3  0 -3  6  7 -6 -7 -6 -6 -4 -3 -6  7
  0  1 -6  7  4  7  7  7]


In [406]:
table_code_words = np.load("table_code_words.npy")
print(table_code_words)

[  0  81 114  35  52 101  70  23 104  57  26  75  92  13  46 127]


In [407]:
table_code_words_binary = np.zeros((16,), dtype=np.ndarray)
j = 0
for i in table_code_words:
    table_code_words_binary[j] = [int(bit) for bit in bin(i)[2:].zfill(7)]
    j += 1

print(table_code_words_binary)

[list([0, 0, 0, 0, 0, 0, 0]) list([1, 0, 1, 0, 0, 0, 1])
 list([1, 1, 1, 0, 0, 1, 0]) list([0, 1, 0, 0, 0, 1, 1])
 list([0, 1, 1, 0, 1, 0, 0]) list([1, 1, 0, 0, 1, 0, 1])
 list([1, 0, 0, 0, 1, 1, 0]) list([0, 0, 1, 0, 1, 1, 1])
 list([1, 1, 0, 1, 0, 0, 0]) list([0, 1, 1, 1, 0, 0, 1])
 list([0, 0, 1, 1, 0, 1, 0]) list([1, 0, 0, 1, 0, 1, 1])
 list([1, 0, 1, 1, 1, 0, 0]) list([0, 0, 0, 1, 1, 0, 1])
 list([0, 1, 0, 1, 1, 1, 0]) list([1, 1, 1, 1, 1, 1, 1])]


# Table hamming distance between code words

In [408]:
table_hamming_distance = np.full((15,15), 255, dtype=np.uint8)
np.fill_diagonal(table_hamming_distance, 0)
print(table_hamming_distance)

[[  0 255 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255   0 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255   0 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255   0 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255   0 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255   0 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255   0 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255   0 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255   0 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255   0 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255   0 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255   0 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255   0]]


In [409]:
for code_word_bin, code_word in zip(table_code_words_binary[1:], table_code_words[1:]):
    err_cor_word = error_correction[code_word] + 7
    for code_word_bin_2, code_word_2 in zip(table_code_words_binary[1:], table_code_words[1:]):
        err_cor_word_2 = error_correction[code_word_2] + 7
        hd = hamming(code_word_bin, code_word_bin_2) * len(code_word_bin)
        table_hamming_distance[err_cor_word][err_cor_word_2] = hd

print(table_hamming_distance)

[[0 3 4 4 3 4 3 4 3 4 3 3 4 7 4]
 [3 0 3 3 4 3 4 3 4 3 4 4 7 4 3]
 [4 3 0 4 3 4 3 4 3 4 3 7 4 3 4]
 [4 3 4 0 3 4 3 4 3 4 7 3 4 3 4]
 [3 4 3 3 0 3 4 3 4 7 4 4 3 4 3]
 [4 3 4 4 3 0 3 4 7 4 3 3 4 3 4]
 [3 4 3 3 4 3 0 7 4 3 4 4 3 4 3]
 [4 3 4 4 3 4 7 0 3 4 3 3 4 3 4]
 [3 4 3 3 4 7 4 3 0 3 4 4 3 4 3]
 [4 3 4 4 7 4 3 4 3 0 3 3 4 3 4]
 [3 4 3 7 4 3 4 3 4 3 0 4 3 4 3]
 [3 4 7 3 4 3 4 3 4 3 4 0 3 4 3]
 [4 7 4 4 3 4 3 4 3 4 3 3 0 3 4]
 [7 4 3 3 4 3 4 3 4 3 4 4 3 0 3]
 [4 3 4 4 3 4 3 4 3 4 3 3 4 3 0]]


# Table min bit flips

In [410]:
# you cant get to -8 so table is 15x15
table_min_bit_flips = np.full((15,15), 255, dtype=np.uint8)
np.fill_diagonal(table_min_bit_flips, 0)
print(table_min_bit_flips)

[[  0 255 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255   0 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255   0 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255   0 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255   0 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255   0 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255   0 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255   0 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255   0 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255   0 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255   0 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255   0 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255   0]]


In [411]:
def bit_flip(bits_arr, word):
    for bit in bits_arr:
        word[bit] = 1 - word[bit]

    return word

def bin_to_dec(bin_arr):
    dec = 0
    for i in range(len(bin_arr)):
        dec += bin_arr[i] * 2 ** (len(bin_arr) - i - 1)

    return np.int8(dec)

## Table min bit flips 1 bit flip

In [412]:
for code_word_bin, code_word in zip(table_code_words_binary[1:], table_code_words[1:]):
    err_cor_word = error_correction[code_word] + 7


    for j in range (0, 7):

        bits = [j]

        bit_flips_word_bin = copy.deepcopy(code_word_bin)
        bit_flips_word_bin = bit_flip(bits, bit_flips_word_bin)
        bit_flips_word = bin_to_dec(bit_flips_word_bin)
        err_cor_bit_flips_word = error_correction[bit_flips_word] + 7

        hd = hamming(bit_flips_word_bin, code_word_bin) * len(code_word_bin)

        # print("code_word: ", code_word_bin)
        # print("err_cor_word: ", err_cor_word)
        # print("bit_flips_word: ", bit_flips_word_bin)
        # print("err_cor_bit_flips_word: ", err_cor_bit_flips_word)
        # print("hd: ", hd)

        if table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] > hd:
            table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] = hd

        # print('-----------------------')




print(table_min_bit_flips)

[[  0 255 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255   0 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255   0 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255   0 255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255   0 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255   0 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255   0 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255   0 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255   0 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255   0 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255   0 255 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255   0 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255   0]]


## Table min bit flips 2 bit flips

In [413]:
error_correction_old = np.load("table_error_correction.npy")

for code_word_bin, code_word in zip(table_code_words_binary[1:], table_code_words[1:]):
    err_cor_word = error_correction[code_word] + 7

    for i in range (0, 7 - 1):
        for j in range (i+1, 7):

            bits = [i, j]

            bit_flips_word_bin = copy.deepcopy(code_word_bin)
            bit_flips_word_bin = bit_flip(bits, bit_flips_word_bin)
            bit_flips_word = bin_to_dec(bit_flips_word_bin)
            err_cor_bit_flips_word = error_correction[bit_flips_word] + 7

            hd = hamming(bit_flips_word_bin, code_word_bin) * len(code_word_bin)

            # print("code_word: ", code_word_bin)
            # print("err_cor_word: ", err_cor_word)
            # print("bit_flips_word: ", bit_flips_word_bin)
            # print("err_cor_bit_flips_word: ", err_cor_bit_flips_word)
            # print("hd: ", hd)


            if table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] > hd:
                table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] = hd
                if (err_cor_bit_flips_word == 0 or err_cor_word == 0) and error_correction_old[bit_flips_word] == -8:
                    print("code_word: ", code_word)
                    print("bit_flips_word: ", bit_flips_word)

                    print("\nerr_cor_word: ", error_correction[code_word])
                    print("err_cor_bit_flips_word: ", error_correction[bit_flips_word])

                    print("\nerr_cor_word_old: ", error_correction_old[code_word])
                    print("err_cor_bit_flips_word_old: ", error_correction_old[bit_flips_word])
                    # print("hd: ", hd)
                    # print("code_word     : ", code_word_bin)
                    # print("bit_flips_word: ", bit_flips_word_bin)
                    # # print(table_min_bit_flips)
                    print('-----------------------')

            # print(table_min_bit_flips)

print(table_min_bit_flips)


code_word:  35
bit_flips_word:  1

err_cor_word:  -5
err_cor_bit_flips_word:  -7

err_cor_word_old:  -5
err_cor_bit_flips_word_old:  -8
-----------------------
code_word:  52
bit_flips_word:  4

err_cor_word:  -4
err_cor_bit_flips_word:  -7

err_cor_word_old:  -4
err_cor_bit_flips_word_old:  -8
-----------------------
code_word:  70
bit_flips_word:  2

err_cor_word:  -2
err_cor_bit_flips_word:  -7

err_cor_word_old:  -2
err_cor_bit_flips_word_old:  -8
-----------------------
code_word:  104
bit_flips_word:  8

err_cor_word:  0
err_cor_bit_flips_word:  -7

err_cor_word_old:  0
err_cor_bit_flips_word_old:  -8
-----------------------
code_word:  26
bit_flips_word:  2

err_cor_word:  2
err_cor_bit_flips_word:  -7

err_cor_word_old:  2
err_cor_bit_flips_word_old:  -8
-----------------------
code_word:  13
bit_flips_word:  1

err_cor_word:  5
err_cor_bit_flips_word:  -7

err_cor_word_old:  5
err_cor_bit_flips_word_old:  -8
-----------------------
[[  0   2 255 255   2 255   2 255   2 255   2

## Table min bit flips 3 bit flips

In [414]:
for code_word_bin, code_word in zip(table_code_words_binary[1:], table_code_words[1:]):
    err_cor_word = error_correction[code_word] + 7

    for k in range (0, 7 - 2):
        for i in range (k+1, 7 - 1):
            for j in range (i+1, 7):

                bits = [k, i, j]

                bit_flips_word_bin = copy.deepcopy(code_word_bin)
                bit_flips_word_bin = bit_flip(bits, bit_flips_word_bin)
                bit_flips_word = bin_to_dec(bit_flips_word_bin)
                err_cor_bit_flips_word = error_correction[bit_flips_word] + 7

                hd = hamming(bit_flips_word_bin, code_word_bin) * len(code_word_bin)

                # print("code_word: ", code_word_bin)
                # print("err_cor_word: ", err_cor_word)
                # print("bit_flips_word: ", bit_flips_word_bin)
                # print("err_cor_bit_flips_word: ", err_cor_bit_flips_word)
                # print("hd: ", hd)

                if table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] > hd:
                    table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] = hd

                # print('-----------------------')




print(table_min_bit_flips)

[[  0   2   3   3   2   3   2   3   2   3   2   2   3 255   3]
 [  2   0   2   2   3   2   3   2   3   2   3   3 255   3   2]
 [  2   2   0   3   2   3   2   3   2   3   2 255   3   2   3]
 [  2   2   3   0   2   3   2   3   2   3 255   2   3   2   3]
 [  2   3   2   2   0   2   3   2   3 255   3   3   2   3   2]
 [  2   2   3   3   2   0   2   3 255   3   2   2   3   2   3]
 [  2   3   2   2   3   2   0 255   3   2   3   3   2   3   2]
 [  2   2   3   3   2   3 255   0   2   3   2   2   3   2   3]
 [  2   3   2   2   3 255   3   2   0   2   3   3   2   3   2]
 [  2   2   3   3 255   3   2   3   2   0   2   2   3   2   3]
 [  2   3   2 255   3   2   3   2   3   2   0   3   2   3   2]
 [  2   3 255   2   3   2   3   2   3   2   3   0   2   3   2]
 [  2 255   3   3   2   3   2   3   2   3   2   2   0   2   3]
 [  3   3   2   2   3   2   3   2   3   2   3   3   2   0   2]
 [  3   2   3   3   2   3   2   3   2   3   2   2   3   2   0]]


# Table min bit flips 4 bit flips

In [415]:
for code_word_bin, code_word in zip(table_code_words_binary[1:], table_code_words[1:]):
    err_cor_word = error_correction[code_word] + 7

    for z in range (0, 7 - 3):
        for k in range (z+1, 7 - 2):
            for i in range (k+1, 7 - 1):
                for j in range (i+1, 7):

                    bits = [z, k, i, j]

                    bit_flips_word_bin = copy.deepcopy(code_word_bin)
                    bit_flips_word_bin = bit_flip(bits, bit_flips_word_bin)
                    bit_flips_word = bin_to_dec(bit_flips_word_bin)
                    err_cor_bit_flips_word = error_correction[bit_flips_word] + 7

                    hd = hamming(bit_flips_word_bin, code_word_bin) * len(code_word_bin)

                    # print("code_word: ", code_word_bin)
                    # print("err_cor_word: ", err_cor_word)
                    # print("bit_flips_word: ", bit_flips_word_bin)
                    # print("err_cor_bit_flips_word: ", err_cor_bit_flips_word)
                    # print("hd: ", hd)

                    if table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] > hd:
                        table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] = hd

                    # print('-----------------------')




print(table_min_bit_flips)

[[  0   2   3   3   2   3   2   3   2   3   2   2   3 255   3]
 [  2   0   2   2   3   2   3   2   3   2   3   3 255   3   2]
 [  2   2   0   3   2   3   2   3   2   3   2 255   3   2   3]
 [  2   2   3   0   2   3   2   3   2   3 255   2   3   2   3]
 [  2   3   2   2   0   2   3   2   3 255   3   3   2   3   2]
 [  2   2   3   3   2   0   2   3 255   3   2   2   3   2   3]
 [  2   3   2   2   3   2   0 255   3   2   3   3   2   3   2]
 [  2   2   3   3   2   3 255   0   2   3   2   2   3   2   3]
 [  2   3   2   2   3 255   3   2   0   2   3   3   2   3   2]
 [  2   2   3   3 255   3   2   3   2   0   2   2   3   2   3]
 [  2   3   2 255   3   2   3   2   3   2   0   3   2   3   2]
 [  2   3 255   2   3   2   3   2   3   2   3   0   2   3   2]
 [  2 255   3   3   2   3   2   3   2   3   2   2   0   2   3]
 [  3   3   2   2   3   2   3   2   3   2   3   3   2   0   2]
 [  3   2   3   3   2   3   2   3   2   3   2   2   3   2   0]]


# Table min bit flips 5 bit flips

In [416]:
for code_word_bin, code_word in zip(table_code_words_binary[1:], table_code_words[1:]):
    err_cor_word = error_correction[code_word] + 7

    for y in range (0, 7 - 4):
        for z in range (y+1, 7 - 3):
            for k in range (z+1, 7 - 2):
                for i in range (k+1, 7 - 1):
                    for j in range (i+1, 7):

                        bits = [y, z, k, i, j]

                        bit_flips_word_bin = copy.deepcopy(code_word_bin)
                        bit_flips_word_bin = bit_flip(bits, bit_flips_word_bin)
                        bit_flips_word = bin_to_dec(bit_flips_word_bin)
                        err_cor_bit_flips_word = error_correction[bit_flips_word] + 7

                        hd = hamming(bit_flips_word_bin, code_word_bin) * len(code_word_bin)

                        # print("code_word: ", code_word_bin)
                        # print("err_cor_word: ", err_cor_word)
                        # print("bit_flips_word: ", bit_flips_word_bin)
                        # print("err_cor_bit_flips_word: ", err_cor_bit_flips_word)
                        # print("hd: ", hd)

                        if table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] > hd:
                            table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] = hd

                        # print('-----------------------')




print(table_min_bit_flips)

[[  0   2   3   3   2   3   2   3   2   3   2   2   3 255   3]
 [  2   0   2   2   3   2   3   2   3   2   3   3 255   3   2]
 [  2   2   0   3   2   3   2   3   2   3   2 255   3   2   3]
 [  2   2   3   0   2   3   2   3   2   3 255   2   3   2   3]
 [  2   3   2   2   0   2   3   2   3 255   3   3   2   3   2]
 [  2   2   3   3   2   0   2   3 255   3   2   2   3   2   3]
 [  2   3   2   2   3   2   0 255   3   2   3   3   2   3   2]
 [  2   2   3   3   2   3 255   0   2   3   2   2   3   2   3]
 [  2   3   2   2   3 255   3   2   0   2   3   3   2   3   2]
 [  2   2   3   3 255   3   2   3   2   0   2   2   3   2   3]
 [  2   3   2 255   3   2   3   2   3   2   0   3   2   3   2]
 [  2   3 255   2   3   2   3   2   3   2   3   0   2   3   2]
 [  2 255   3   3   2   3   2   3   2   3   2   2   0   2   3]
 [  3   3   2   2   3   2   3   2   3   2   3   3   2   0   2]
 [  3   2   3   3   2   3   2   3   2   3   2   2   3   2   0]]


# Table min bit flips 6 bit flips

In [417]:
for code_word_bin, code_word in zip(table_code_words_binary[1:], table_code_words[1:]):
    err_cor_word = error_correction[code_word] + 7

    for x in range (0, 7 - 5):
        for y in range (x+1, 7 - 4):
            for z in range (y+1, 7 - 3):
                for k in range (z+1, 7 - 2):
                    for i in range (k+1, 7 - 1):
                        for j in range (i+1, 7):

                            bits = [x, y, z, k, i, j]

                            bit_flips_word_bin = copy.deepcopy(code_word_bin)
                            bit_flips_word_bin = bit_flip(bits, bit_flips_word_bin)
                            bit_flips_word = bin_to_dec(bit_flips_word_bin)
                            err_cor_bit_flips_word = error_correction[bit_flips_word] + 7

                            hd = hamming(bit_flips_word_bin, code_word_bin) * len(code_word_bin)

                            # print("code_word: ", code_word_bin)
                            # print("err_cor_word: ", err_cor_word)
                            # print("bit_flips_word: ", bit_flips_word_bin)
                            # print("err_cor_bit_flips_word: ", err_cor_bit_flips_word)
                            # print("hd: ", hd)

                            if table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] > hd:
                                table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] = hd

                            # print('-----------------------')




print(table_min_bit_flips)

[[0 2 3 3 2 3 2 3 2 3 2 2 3 6 3]
 [2 0 2 2 3 2 3 2 3 2 3 3 6 3 2]
 [2 2 0 3 2 3 2 3 2 3 2 6 3 2 3]
 [2 2 3 0 2 3 2 3 2 3 6 2 3 2 3]
 [2 3 2 2 0 2 3 2 3 6 3 3 2 3 2]
 [2 2 3 3 2 0 2 3 6 3 2 2 3 2 3]
 [2 3 2 2 3 2 0 6 3 2 3 3 2 3 2]
 [2 2 3 3 2 3 6 0 2 3 2 2 3 2 3]
 [2 3 2 2 3 6 3 2 0 2 3 3 2 3 2]
 [2 2 3 3 6 3 2 3 2 0 2 2 3 2 3]
 [2 3 2 6 3 2 3 2 3 2 0 3 2 3 2]
 [2 3 6 2 3 2 3 2 3 2 3 0 2 3 2]
 [2 6 3 3 2 3 2 3 2 3 2 2 0 2 3]
 [3 3 2 2 3 2 3 2 3 2 3 3 2 0 2]
 [3 2 3 3 2 3 2 3 2 3 2 2 3 2 0]]


# Table min bit flips 7 bit flips

In [418]:
for code_word_bin, code_word in zip(table_code_words_binary[1:], table_code_words[1:]):
    err_cor_word = error_correction[code_word] + 7

    for w in range (0, 7 - 6):
        for x in range (w+1, 7 - 5):
            for y in range (x+1, 7 - 4):
                for z in range (y+1, 7 - 3):
                    for k in range (z+1, 7 - 2):
                        for i in range (k+1, 7 - 1):
                            for j in range (i+1, 7):

                                bits = [w, x, y, z, k, i, j]

                                bit_flips_word_bin = copy.deepcopy(code_word_bin)
                                bit_flips_word_bin = bit_flip(bits, bit_flips_word_bin)
                                bit_flips_word = bin_to_dec(bit_flips_word_bin)
                                err_cor_bit_flips_word = error_correction[bit_flips_word] + 7

                                hd = hamming(bit_flips_word_bin, code_word_bin) * len(code_word_bin)

                                # print("code_word: ", code_word_bin)
                                # print("err_cor_word: ", err_cor_word)
                                # print("bit_flips_word: ", bit_flips_word_bin)
                                # print("err_cor_bit_flips_word: ", err_cor_bit_flips_word)
                                # print("hd: ", hd)

                                if table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] > hd:
                                    table_min_bit_flips[err_cor_word][err_cor_bit_flips_word] = hd

                                # print('-----------------------')




print(table_min_bit_flips)

[[0 2 3 3 2 3 2 3 2 3 2 2 3 6 3]
 [2 0 2 2 3 2 3 2 3 2 3 3 6 3 2]
 [2 2 0 3 2 3 2 3 2 3 2 6 3 2 3]
 [2 2 3 0 2 3 2 3 2 3 6 2 3 2 3]
 [2 3 2 2 0 2 3 2 3 6 3 3 2 3 2]
 [2 2 3 3 2 0 2 3 6 3 2 2 3 2 3]
 [2 3 2 2 3 2 0 6 3 2 3 3 2 3 2]
 [2 2 3 3 2 3 6 0 2 3 2 2 3 2 3]
 [2 3 2 2 3 6 3 2 0 2 3 3 2 3 2]
 [2 2 3 3 6 3 2 3 2 0 2 2 3 2 3]
 [2 3 2 6 3 2 3 2 3 2 0 3 2 3 2]
 [2 3 6 2 3 2 3 2 3 2 3 0 2 3 2]
 [2 6 3 3 2 3 2 3 2 3 2 2 0 2 3]
 [3 3 2 2 3 2 3 2 3 2 3 3 2 0 2]
 [3 2 3 3 2 3 2 3 2 3 2 2 3 2 0]]


In [419]:
print(table_hamming_distance)

[[0 3 4 4 3 4 3 4 3 4 3 3 4 7 4]
 [3 0 3 3 4 3 4 3 4 3 4 4 7 4 3]
 [4 3 0 4 3 4 3 4 3 4 3 7 4 3 4]
 [4 3 4 0 3 4 3 4 3 4 7 3 4 3 4]
 [3 4 3 3 0 3 4 3 4 7 4 4 3 4 3]
 [4 3 4 4 3 0 3 4 7 4 3 3 4 3 4]
 [3 4 3 3 4 3 0 7 4 3 4 4 3 4 3]
 [4 3 4 4 3 4 7 0 3 4 3 3 4 3 4]
 [3 4 3 3 4 7 4 3 0 3 4 4 3 4 3]
 [4 3 4 4 7 4 3 4 3 0 3 3 4 3 4]
 [3 4 3 7 4 3 4 3 4 3 0 4 3 4 3]
 [3 4 7 3 4 3 4 3 4 3 4 0 3 4 3]
 [4 7 4 4 3 4 3 4 3 4 3 3 0 3 4]
 [7 4 3 3 4 3 4 3 4 3 4 4 3 0 3]
 [4 3 4 4 3 4 3 4 3 4 3 3 4 3 0]]


# IT doesn't add up because of the -8 we are pushing it to -7 hence the difference in first column

In [420]:
print(table_hamming_distance - table_min_bit_flips)

[[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 1 1 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
 [2 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
 [2 1 1 1 1 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]
 [2 1 1 1 1 1 1 1 1 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]
 [2 1 1 1 1 1 1 1 1 1 1 1 0 1 1]
 [4 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]]


# Save the table

In [421]:
# np.save('table_min_bit_flips.npy', table_min_bit_flips)